In [1]:
import numpy as np

index = np.load("/home/jovyan/Sample_Based_Extension/UNSW/best_defense_indices.npy", allow_pickle=True)
index

array([['baseline', 0, 'Def1-11 Combined', 0.9923313825446824,
        0.9911800925969834, array([9, 3, 9, ..., 9, 9, 9])],
       ['BIM', 0.01, 'Def1-11 Combined', 0.9891551359556379,
        0.987484040917074, array([9, 3, 9, ..., 9, 9, 9])],
       ['BIM', 0.1, 'Def1-11 Combined', 0.9746251239350361,
        0.9705688736459472, array([1, 6, 1, ..., 1, 1, 1])],
       ['BIM', 0.2, 'Def1-11 Combined', 0.9594545980995165,
        0.9538946564599001, array([6, 6, 6, ..., 6, 6, 6])],
       ['BIM', 0.3, 'Def1-11 Combined', 0.919233840187416,
        0.9113183206844415, array([6, 6, 6, ..., 6, 6, 6])],
       ['FGSM', 0.01, 'Def1-11 Combined', 0.9889357819094331,
        0.9872271983425641, array([9, 3, 9, ..., 9, 9, 9])],
       ['FGSM', 0.1, 'Def1-11 Combined', 0.9680708250344385,
        0.9628361221532343, array([1, 6, 1, ..., 1, 1, 1])],
       ['FGSM', 0.2, 'Def1-11 Combined', 0.9461968395469023,
        0.9384213027985543, array([1, 6, 1, ..., 1, 1, 1])],
       ['FGSM', 0.3, 'Def1

In [2]:
attack_models = [
    ("BIM", [0.1]),
    ("FGSM", [0.1]),
    ("PGD", [0.1]),
    ("DF", [0.1]),
    ("AutoPGD", [0.1]),
    ("ZOO", [0.1]),
    ("CaFA", [0.1]),
    ("SINIFGSM", [0.1]),
    ("VNIFGSM", [0.1]),
]

all_adv_examples = []

for attack_name, epsilons in attack_models:
    for epsilon in epsilons:
        filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
        x_test_adv = np.load(filename)
        all_adv_examples.append(x_test_adv)

x_test_adv_0_1 = np.concatenate(all_adv_examples, axis=0)


In [3]:
y_labels = []

for row in index:
    attack_name, eps = row[0], row[1]
    if eps == 0.1:
        y_array = row[-1]
        y_labels.append(y_array)

y_label_0_1 = np.concatenate(y_labels, axis=0)

In [4]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_label_0_1)

y_label_0_1 = label_encoder.transform(y_label_0_1)

for index, class_label in enumerate(label_encoder.classes_):
    print(f"defense '{class_label}' encode {index}")

defense '1' encode 0
defense '2' encode 1
defense '3' encode 2
defense '4' encode 3
defense '5' encode 4
defense '6' encode 5
defense '7' encode 6
defense '8' encode 7
defense '9' encode 8
defense '10' encode 9
defense '11' encode 10


In [6]:
from xgboost import XGBClassifier  # 替换为 XGBoost
from sklearn.metrics import accuracy_score
from sklearn.cluster import MiniBatchKMeans
import numpy as np
from scipy.sparse import vstack

def stratified_random_sample(X, y, fraction=None, count=None, random_state=42):
    assert (fraction is not None) ^ (count is not None), "Specify either fraction or count"

    rng = np.random.RandomState(seed=random_state)
    X_out, y_out = [], []

    for label in np.unique(y):
        X_c = X[y == label]
        y_c = y[y == label]

        n_total = len(X_c)
        n_sample = int(n_total * fraction) if fraction is not None else min(count, n_total)
        if n_sample == 0:
            n_sample = 1

        idx = rng.choice(n_total, size=n_sample, replace=False)

        X_out.append(X_c[idx])
        y_out.append(y_c[idx])

    return np.concatenate(X_out, axis=0), np.concatenate(y_out, axis=0)


def batch_mode_active_learning(X_pool, y_pool, model, fraction=0.2, random_state=42):
    X_selected = []
    y_selected = []
    selected_indices = []

    rng = np.random.RandomState(seed=random_state)
    n_total = len(X_pool)
    n_select = max(1, int(n_total * fraction))
    classes = np.unique(y_pool)

    # Get uncertainties
    probs = model.predict_proba(X_pool)
    uncertainties = 1 - np.max(probs, axis=1)

    # Ensure at least one sample per class
    min_samples_per_class = 1
    n_guaranteed = min_samples_per_class * len(classes)
    n_clusters = max(1, n_select - n_guaranteed)  # Reserve clusters for remaining samples

    # Step 1: Select one sample per class based on highest uncertainty
    for c in classes:
        idx_c = np.where(y_pool == c)[0]
        if len(idx_c) == 0:
            continue
        uncertainties_c = uncertainties[idx_c]
        top_idx = idx_c[np.argmax(uncertainties_c)]  # Select sample with max uncertainty
        selected_indices.append(top_idx)

    # Step 2: Select remaining samples using batch mode active learning
    remaining_idx = np.setdiff1d(np.arange(len(X_pool)), selected_indices)
    if n_clusters > 0 and len(remaining_idx) > 0:
        X_remaining = X_pool[remaining_idx]
        uncertainties_remaining = uncertainties[remaining_idx]

        # Select top-k uncertain instances for clustering
        top_k = min(n_clusters * 10, len(remaining_idx))
        top_indices = remaining_idx[np.argsort(uncertainties_remaining)[-top_k:]]

        # Cluster with MiniBatchKMeans
        kmeans = MiniBatchKMeans(n_clusters=n_clusters, random_state=random_state, batch_size=1000, max_iter=100)
        kmeans.fit(X_pool[top_indices])

        # Select the instance closest to each cluster center
        for center in kmeans.cluster_centers_:
            distances = np.linalg.norm(X_pool[top_indices] - center, axis=1)
            selected_indices.append(top_indices[np.argmin(distances)])

    selected_indices = np.array(selected_indices)
    X_selected = X_pool[selected_indices]
    y_selected = y_pool[selected_indices]

    # Remaining data
    remaining_idx = np.setdiff1d(np.arange(len(X_pool)), selected_indices)
    X_rest = X_pool[remaining_idx]
    y_rest = y_pool[remaining_idx]

    # Print selected samples' label distribution
    print(f"Selected samples (fraction={fraction}): {len(y_selected)}")
    print(f"Label distribution: {np.bincount(y_selected, minlength=11)}")

    return X_rest, X_selected, y_rest, y_selected


# Example usage
X_pool = x_test_adv_0_1  # Your data (ensure dense NumPy array)
y_pool = y_label_0_1     # Your labels (0 to 10)

# Convert sparse to dense if necessary
if hasattr(X_pool, 'toarray'):
    X_pool = X_pool.toarray()

# Initial stratified sampling
X_train_init, y_train_init = stratified_random_sample(X_pool, y_pool, fraction=0.1, random_state=42)
print(f"Initial labeled set size: {len(y_train_init)}")
print(f"Initial label distribution: {np.bincount(y_train_init, minlength=11)}")

# Train initial model with XGBoost
clf = XGBClassifier(random_state=42)  # Use GPU acceleration
clf.fit(X_train_init, y_train_init)

# Perform batch mode active learning with fractions 50%, 20%, 1%

_, X_1, _, y_1 = batch_mode_active_learning(
    x_test_adv_0_1, y_label_0_1, clf, fraction=0.01, random_state=42
)

Initial labeled set size: 102570
Initial label distribution: [75542   161   700  1743   377 10542    74   736 12435   172    88]
Selected samples (fraction=0.01): 10257
Label distribution: [4043   86  352  573  119  979    5  131 3857   83   29]


In [7]:
X_1.shape

(10257, 56)

In [8]:
from xgboost import XGBClassifier

print('xgb')
xgb1 = XGBClassifier()
xgb1.fit(X_1, y_1)

print("RF")
from sklearn.ensemble import RandomForestClassifier
rf1 = RandomForestClassifier()
rf1.fit(X_1, y_1)

print("DT")
from sklearn.tree import DecisionTreeClassifier
dt1 = DecisionTreeClassifier()
dt1.fit(X_1, y_1)

xgb
RF
DT


DecisionTreeClassifier()

In [9]:
import time
import numpy as np

y_test = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

attack_models = [
    ("baseline", [0]),
    ("BIM", [0.01, 0.2, 0.3]),
    ("FGSM", [0.01, 0.2, 0.3]),
    ("PGD", [0.01, 0.2, 0.3]),
    ("DF", [0.01, 0.2, 0.3]),
    ("AutoPGD", [0.01, 0.2, 0.3]),
    ("ZOO", [0.01, 0.2, 0.3]),
    ("CaFA", [0.01, 0.2, 0.3]),
    ("SINIFGSM", [0.01, 0.2, 0.3]),
    ("VNIFGSM", [0.01, 0.2, 0.3]),
]

base_path_template = "/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack}_eps_{eps}.npy"

from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

    y_pred = xgb1.predict(X)
    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/XGB_ActiveLearning_BatchMode/UNSW_Input1/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/XGB_ActiveLearning_BatchMode/UNSW_Input1/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  4  5  6  7  8  9 10] [54144   138  1546  1150    34  3789     1    89 52968    87    25]
(54144, 56) (54144,)
Save 0 to baseline_Def1.npy
(138, 56) (138,)
Save 1 to baseline_Def2.npy
(1546, 56) (1546,)
Save 2 to baseline_Def3.npy
(1150, 56) (1150,)
Save 3 to baseline_Def4.npy
(34, 56) (34,)
Save 4 to baseline_Def5.npy
(3789, 56) (3789,)
Save 5 to baseline_Def6.npy
(1, 56) (1,)
Save 6 to baseline_Def7.npy
(89, 56) (89,)
Save 7 to baseline_Def8.npy
(52968, 56) (52968,)
Save 8 to baseline_Def9.npy
(87, 56) (87,)
Save 9 to baseline_Def10.npy
(25, 56) (25,)
Save 10 to baseline_Def11.npy
Execution Time: 0.835109 seconds
(341913,)
[ 0  1  2  3  4  5  6  7  8  9 10] [191411    176   1126   3332   2788  86396      1    255  56330     83
     15]
(191411, 56) (191411,)
Save 0 to BIM_Def1.npy
(176, 56) (176,)
Save 1 to BIM_Def2.npy
(1126, 56) (1126,)
Save 2 to BIM_Def3.npy
(3332, 56) (3332,)
Save 3 to BIM_Def4.npy
(2788, 56) (2788,)
Save 4 to BIM_Def5.npy
(86396, 56) (8639

In [10]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

    y_pred = rf1.predict(X)
    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_ActiveLearning_BatchMode/UNSW_Input1/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_ActiveLearning_BatchMode/UNSW_Input1/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  4  5  6  7  8  9 10] [52866   175  1301   941    28  4116     1    83 54391    50    19]
(52866, 56) (52866,)
Save 0 to baseline_Def1.npy
(175, 56) (175,)
Save 1 to baseline_Def2.npy
(1301, 56) (1301,)
Save 2 to baseline_Def3.npy
(941, 56) (941,)
Save 3 to baseline_Def4.npy
(28, 56) (28,)
Save 4 to baseline_Def5.npy
(4116, 56) (4116,)
Save 5 to baseline_Def6.npy
(1, 56) (1,)
Save 6 to baseline_Def7.npy
(83, 56) (83,)
Save 7 to baseline_Def8.npy
(54391, 56) (54391,)
Save 8 to baseline_Def9.npy
(50, 56) (50,)
Save 9 to baseline_Def10.npy
(19, 56) (19,)
Save 10 to baseline_Def11.npy
Execution Time: 1.493289 seconds
(341913,)
[0 1 2 3 4 5 7 8 9] [241462     16   1124   2393    204  52684    121  43906      3]
(241462, 56) (241462,)
Save 0 to BIM_Def1.npy
(16, 56) (16,)
Save 1 to BIM_Def2.npy
(1124, 56) (1124,)
Save 2 to BIM_Def3.npy
(2393, 56) (2393,)
Save 3 to BIM_Def4.npy
(204, 56) (204,)
Save 4 to BIM_Def5.npy
(52684, 56) (52684,)
Save 5 to BIM_Def6.npy
(121, 56)

In [11]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

    y_pred = dt1.predict(X)
    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/DT_ActiveLearning_BatchMode/UNSW_Input1/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/DT_ActiveLearning_BatchMode/UNSW_Input1/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  4  5  6  7  8  9 10] [47137   555  2896  2191   109 10328     1   846 48515  1043   350]
(47137, 56) (47137,)
Save 0 to baseline_Def1.npy
(555, 56) (555,)
Save 1 to baseline_Def2.npy
(2896, 56) (2896,)
Save 2 to baseline_Def3.npy
(2191, 56) (2191,)
Save 3 to baseline_Def4.npy
(109, 56) (109,)
Save 4 to baseline_Def5.npy
(10328, 56) (10328,)
Save 5 to baseline_Def6.npy
(1, 56) (1,)
Save 6 to baseline_Def7.npy
(846, 56) (846,)
Save 7 to baseline_Def8.npy
(48515, 56) (48515,)
Save 8 to baseline_Def9.npy
(1043, 56) (1043,)
Save 9 to baseline_Def10.npy
(350, 56) (350,)
Save 10 to baseline_Def11.npy
Execution Time: 0.202812 seconds
(341913,)
[ 0  1  2  3  4  5  7  8  9 10] [69135  1910 15865 84475  6453 65179 13328 68973 15435  1160]
(69135, 56) (69135,)
Save 0 to BIM_Def1.npy
(1910, 56) (1910,)
Save 1 to BIM_Def2.npy
(15865, 56) (15865,)
Save 2 to BIM_Def3.npy
(84475, 56) (84475,)
Save 3 to BIM_Def4.npy
(6453, 56) (6453,)
Save 4 to BIM_Def5.npy
(65179, 56) (65179,)
S

In [12]:
import numpy as np


attack_names = [
    "baseline", 
    "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

all_attack_list = []

model_name_list = ["XGB", "RF", "DT"]

for m_name in model_name_list:
    for attack_name in attack_names:
        each = []
        num = 0
        each.append(f"{m_name}1")
        for i in range(1, 12):
            y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/{m_name}_ActiveLearning_BatchMode/UNSW_Input1/x_test_adv_{attack_name}_Def{i}.npy"
            try:
                y = np.load(y_filename)
                num+=y.shape[0]
                each.append(y.shape[0])
            except FileNotFoundError:
                each.append(0)
        each.append(num)
        all_attack_list.append(each)


In [13]:
import pandas as pd
all_attack_list = pd.DataFrame(all_attack_list)
all_attack_list.to_csv("./Dynamite_Num_Attack_Input1_ActiveLearning_BatchMode.csv")

In [14]:
all_attack_list

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,XGB1,54144,138,1546,1150,34,3789,1,89,52968,87,25,113971
1,XGB1,191411,176,1126,3332,2788,86396,1,255,56330,83,15,341913
2,XGB1,177313,66,1149,5209,869,86043,1,16829,54286,114,34,341913
3,XGB1,191411,176,1126,3332,2788,86396,1,255,56330,83,15,341913
4,XGB1,168091,1363,12815,620,46,74608,26098,47945,9801,518,8,341913
5,XGB1,209974,308,3651,5162,1984,78650,2,228,41705,180,69,341913
6,XGB1,166929,376,4680,3370,108,12848,3,267,153010,247,75,341913
7,XGB1,176044,37,413,5790,34,82952,1,52,73122,3241,227,341913
8,XGB1,222223,235,843,1811,2136,73157,1,193,41111,184,19,341913
9,XGB1,207240,102,1485,4186,1516,77071,1,267,49838,183,24,341913
